# 1. Librerias

In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
import pandas as pd
import os
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import pandas as pd
from datetime import datetime

# 2. Descargar la base de datos

In [125]:
### DECARGAR DATAS ###

# URLs correctas (versión raw)
url_data_final = "https://raw.githubusercontent.com/CientificoDatos-Lagobo/Agente_simuldor_libranza/main/3.Subir_datos/1.Base_capital/data/Data_final.xlsx"
url_fecha_pagaduria = "https://raw.githubusercontent.com/CientificoDatos-Lagobo/Agente_simuldor_libranza/main/3.Subir_datos/2.Fechas_pagadurias/Fecha_pagaduria.xlsx"
url_tasa_usura = "https://raw.githubusercontent.com/CientificoDatos-Lagobo/Agente_simuldor_libranza/main/3.Subir_datos/3.Tasa_usura/Tasa_usura.xlsx"


try:
    # Leer archivos
    df_data_final = pd.read_excel(url_data_final)
    data_pagaduria = pd.read_excel(url_fecha_pagaduria)
    df_tasa_usura = pd.read_excel(url_tasa_usura)

    # Mensaje de éxito
    print("✅ Archivos descargados y leídos correctamente.")
except Exception as e:
    # Mensaje de error
    print(f"❌ Error al descargar o leer los archivos: {e}")

✅ Archivos descargados y leídos correctamente.


# 3. Obtener informacion inicial

In [126]:
nro_libranza = 43827

In [127]:
### FILTRAR DATOS DEL CREDITO A SMILULAR ###

try:
    # Filtrar datos del credito específico
    data = df_data_final[df_data_final["nolibra"] == nro_libranza]

    # Mensaje de éxito
    print("✅ Datos del credito filtrados correctamente.")
except Exception as e:
    # Mensaje de error
    print(f"❌ Error al filtrar los datos del credito: {e}")


✅ Datos del credito filtrados correctamente.


In [128]:
### VARIABLES Y PARAMETROS PARA LA SIMULACION ###

try:

    # Obtener fecha actual
    hoy = datetime.today()

    # Obtener pagaduría según libranza
    pagaduria = data.loc[data["nolibra"] == nro_libranza, "pagaduria"].values[0]

    # Monto total del préstamo
    monto_prestado = data.loc[data["nolibra"] == nro_libranza, "base_capital"].values[0]

    # Tasa mensual nominal (decimal)
    tasa_mensual = data.loc[data["nolibra"] == nro_libranza, "tasa_credito"].values[0]

    # Plazo en meses
    plazo_meses = data.loc[data["nolibra"] == nro_libranza, "plazo"].values[0]
    plazo = plazo_meses  # para usar un nombre corto

    # Seguro deudor mensual
    seguro_deudor = data.loc[data["nolibra"] == nro_libranza, "seguro_deudores"].values[0] / plazo_meses

    # Fecha inicio crédito (datetime)
    fecha1 = pd.to_datetime(data.loc[data["nolibra"] == nro_libranza, "fecha_inicio_credito"].values[0])
    fecha2 = pd.to_datetime(data.loc[data["nolibra"] == nro_libranza, "fecha_primer_pago"].values[0])
    fecha_inicio = min(fecha1, fecha2) # Asegurar que sea la fecha más temprana

    # Saldo cierre mes 0
    saldo_cierre_mes_0 = data.loc[data["nolibra"] == nro_libranza, "saldo_cierre_mes_0"].values[0]

    # Cuotas pagas mes
    cuotas_paga_mes = data.loc[data["nolibra"] == nro_libranza, "cuotas_pagas"].values[0]

    # Cuota definida
    cuotadef = data.loc[data["nolibra"] == nro_libranza, "cuota_definida"].values[0]

    # Mensaje
    print(f"✅ Variables de entorno obtenidas correctamente.")

except Exception as e:
    # Mensaje de error
    print(f"❌ Error al obtener las variables de entorno: {e}")

✅ Variables de entorno obtenidas correctamente.


### OJOOOOOOOOOOOOOOOOO REVISAR ####

if hoy.date() > fecha_programada.date():           # Verificar si hoy paso la fecha programada
ESTA ES LA QUE FUNCIONA


if hoy.date() >= fecha_programada.date():           # Verificar si hoy paso la fecha programada
ESTA ES LA PROPUESTA



In [129]:
### CALCULAR FECHA DE CORTE DEL SALDO DEL SISTEMA ###

try:
    # Convertir la fecha de hoy a un objeto datetime
    data_pagaduria["FECHA"] = pd.to_datetime(data_pagaduria["FECHA"])

    # Obtener fecha programada de pagaduría
    fecha_programada = pd.to_datetime(data_pagaduria.loc[data_pagaduria["PAGADURIA"] == pagaduria, "FECHA"].values[0])

    # Calcular corte_saldo_sistema
    #if hoy.date() >= fecha_programada.date():           # Verificar si hoy paso la fecha programada
    if hoy.date() > fecha_programada.date():           # Verificar si hoy paso la fecha programada
        año, mes = hoy.year, hoy.month                 # Si cumple, usar año y mes actual
    else:
        año, mes = (hoy.year - 1, 12) if hoy.month == 1 else (hoy.year, hoy.month - 1)   # Si no ha pasado, usar mes anterior

    ultimo_dia = calendar.monthrange(año, mes)[1]                                        # Último día del mes calculado
    corte_saldo_sistema = datetime(año, mes, ultimo_dia)                                 # Fecha de corte: último día del mes

    # Mensaje
    print(f"✅ Fecha de corte del saldo del sistema calculada: {corte_saldo_sistema.strftime('%Y-%m-%d')}")
    
except Exception as e:
    # Mensaje de error
    print(f"❌ Error al calcular la fecha de corte del saldo del sistema: {e}")

✅ Fecha de corte del saldo del sistema calculada: 2025-08-31


In [130]:
### CALCULAR CUOTAS CAUSADAS PARA LA FECHA DE CORTE DEL SALDO DEL SISTEMA ###

try:

    # Cuotas causadas calculadas
    diff = relativedelta(corte_saldo_sistema, pd.to_datetime(data.loc[data["nolibra"] == nro_libranza, "fecha_inicio_credito"].values[0]))
    cuotas_causadas = diff.years * 12 + diff.months + 1

    print(f"✅ Cuotas causadas calculadas: {cuotas_causadas}")
except Exception as e:
    # Mensaje de error
    print(f"❌ Error al calcular las cuotas causadas: {e}")

✅ Cuotas causadas calculadas: 47


# 4. Desarrollar simulador

In [131]:
### INFORMACION INICIAL DEL SIMULADOR ###

try:

    # Fechas de pago (al final de cada mes)
    fechas_pago = pd.date_range(start=fecha_inicio, periods=plazo, freq="M")

    # Cuota fija mensual (fórmula francesa)
    cuota = monto_prestado * (tasa_mensual * (1 + tasa_mensual) ** plazo) / ((1 + tasa_mensual) ** plazo - 1)

    # Saldo inicial
    saldo = monto_prestado

    # Inicializar tabla amortización con fila saldo inicial
    tabla = [{
        "N°": 0,
        "FECHA": "",
        "TASA LIQUIDACION": "",
        "CUOTA": "",
        "SEGURO DEUDORES": round(seguro_deudor),
        "INTERES": "",
        "AMORTIZACION": "",
        "SALDO": round(saldo)
    }]

    # Mensaje
    print("✅ Simulador configurado correctamente.")

except Exception as e:
    # Mensaje de error
    print(f"❌ Error al configurar el simulador: {e}")

✅ Simulador configurado correctamente.


C:\Users\cientifico.datos\AppData\Local\Temp\ipykernel_12080\2233916589.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fechas_pago = pd.date_range(start=fecha_inicio, periods=plazo, freq="M")


In [132]:
### REALIZAR SIMULACION DE AMORTIZACION ###   

try:
    for i in range(int(plazo_meses)):
        interes = saldo * tasa_mensual
        amortizacion = cuota - interes

        if i == plazo_meses - 1:
            amortizacion = saldo
            cuota = interes + amortizacion

        saldo -= amortizacion

        fila = {
            "N°": i + 1,
            "FECHA": fechas_pago[i].strftime("%d/%m/%Y"),
            "TASA LIQUIDACION": f"{tasa_mensual * 100:.5f}%",
            "CUOTA": round(cuota),
            "SEGURO DEUDORES": round(seguro_deudor),
            "INTERES": round(interes),
            "AMORTIZACION": round(amortizacion),
            "SALDO": round(max(saldo, 0))
        }
        tabla.append(fila)

    # Convertir a DataFrame
    df = pd.DataFrame(tabla)
    
    # Convertir FECHA a datetime para merge posterior
    df["FECHA"] = pd.to_datetime(df["FECHA"], format="%d/%m/%Y", errors="coerce")

    # Mensaje
    print("✅ Tabla de amortización generada correctamente.")
except Exception as e:
    # Mensaje de error
    print(f"❌ Error al generar la tabla de amortización: {e}")

✅ Tabla de amortización generada correctamente.


In [133]:
### CARGAR DATOS DE TASA USURA ###

try:
    # Convertir fechas a datetime
    df_tasa_usura["Fecha Inicio"] = pd.to_datetime(df_tasa_usura["Fecha Inicio"])
    df_tasa_usura["Fecha Fin"] = pd.to_datetime(df_tasa_usura["Fecha Fin"])

    # Expandir rangos de fechas para usura (un día por fila)
    rangos_usura = []
    for _, fila in df_tasa_usura.iterrows():
        fechas = pd.date_range(start=fila["Fecha Inicio"], end=fila["Fecha Fin"], freq="D")
        for fecha in fechas:
            rangos_usura.append({"FECHA": fecha, "TASA_USURA": fila["TASA_USURA"]})

    # Crear DataFrame de rangos de usura        
    df_rangos_usura = pd.DataFrame(rangos_usura)

    # Mensaje
    print("✅ Datos de tasa usura descargados correctamente.")

except Exception as e:
    # Mensaje de error
    print(f"❌ Error al descargar los datos de tasa usura: {e}")

✅ Datos de tasa usura descargados correctamente.


In [134]:
### UNIR DATOS DE TASA USURA AL DATA FRAMDE LA SIMULACION ###

try:
    # Unir con tasa de usura y reordenar columnas
    df = df.merge(df_rangos_usura, on="FECHA", how="left")

    # Rellenar NaN en TASA_USURA con 0
    df["TASA_USURA"] = df["TASA_USURA"].fillna(0)

    # Reordenar para que tasa_usura quede justo después de tasa_liquidacion
    columnas = df.columns.tolist()
    posicion = columnas.index("TASA LIQUIDACION") + 1
    columnas.insert(posicion, columnas.pop(columnas.index("TASA_USURA")))
    df = df[columnas]

    # Mensaje
    print("✅ Datos de tasa usura unidos correctamente.")
except Exception as e:
    # Mensaje de error
    print(f"❌ Error al unir los datos de tasa usura: {e}")

✅ Datos de tasa usura unidos correctamente.


In [135]:
### ARMAR NUEVA TABLA DE AMORTIZACION CON LA TASA DE USURA ###

try:
    # Crear una copia del DataFrame original   
    df_ajustado = df.copy()

    # Convertir tasa_liquidacion a decimal
    df_ajustado["TASA_LIQUIDACION_VALOR"] = (
        pd.to_numeric(
            df_ajustado["TASA LIQUIDACION"].str.replace("%", "", regex=False).str.replace(",", "."),
            errors="coerce"
        ) / 100
    )

    # Calcular tasa aplicada (mínimo entre tasa_liq y tasa_usura si existe)
    df_ajustado["TASA APLICADA"] = df_ajustado.apply(
        lambda fila: fila["TASA_LIQUIDACION_VALOR"]
        if pd.isna(fila.get("TASA_USURA", float("nan"))) or fila.get("TASA_USURA", 0) == 0
        else min(fila["TASA_LIQUIDACION_VALOR"], fila["TASA_USURA"]),
        axis=1
    )

    # Obtener cuota de fila 1 para usar después
    cuota = df_ajustado.loc[1, "CUOTA"] if 1 in df_ajustado.index and "CUOTA" in df_ajustado.columns else 0

    # Llamar variables inici
    saldo = monto_prestado
    acumulado_interes = 0

    # Calcular saldo inicial
    def formatear(numero, decimales=0):
        if pd.isna(numero):
            return ""
        return f"{numero:,.{decimales}f}".replace(",", "X").replace(".", ",").replace("X", ".")
    
    # Inicializar lista de filas para el DataFrame final
    filas = []
    filas.append({
        "Nº": 0,
        "FECHA": "",
        "TASA_LIQUIDACIÓN": "",
        "TASA DE USURA": "",
        "SEGURO DEUDORES": "",
        "FINANCIACIÓN CAPITAL": "",
        "AMORTIZACIÓN CAPITAL": "",
        "SALDO CAPITAL": formatear(saldo, 0),
        "Acumulado_financiacion_capital": ""
    })

    # Recorrer el plazo completo
    for i in range(1, round(plazo) + 1):
        f = df_ajustado.iloc[i]
        tasa_liq = f["TASA_LIQUIDACION_VALOR"]
        tasa_usura = f.get("TASA_USURA", float("nan"))
        tasa_aplicada = f["TASA APLICADA"]
        interes = saldo * tasa_aplicada
        amort = cuota - interes
        saldo -= amort
        acumulado_interes += interes

        # Manejo de fecha
        fecha_str = ""
        if "FECHA" in f and pd.notna(f["FECHA"]):
            try:
                fecha_str = pd.to_datetime(f["FECHA"]).strftime("%d/%m/%Y")
            except Exception:
                fecha_str = str(f["FECHA"])
        filas.append({
            "Nº": int(f["N°"]) if "N°" in f and pd.notna(f["N°"]) else i,
            "FECHA": fecha_str,
            "TASA_LIQUIDACIÓN": formatear(tasa_liq * 100, 5) + "%" if pd.notna(tasa_liq) else "",
            "TASA DE USURA": "" if pd.isna(tasa_usura) or tasa_usura == 0 else formatear(tasa_usura * 100, 6) + "%",
            "SEGURO DEUDORES": formatear(f.get("SEGURO DEUDORES", 0), 0),
            "FINANCIACIÓN CAPITAL": formatear(interes, 0),
            "AMORTIZACIÓN CAPITAL": formatear(amort, 0),
            "SALDO CAPITAL": formatear(saldo, 0),
            "Acumulado_financiacion_capital": formatear(acumulado_interes, 0)
        })

    # Construir el DataFrame final
    df_final = pd.DataFrame(filas)[[
        "Nº", "FECHA", "TASA_LIQUIDACIÓN", "TASA DE USURA",
        "SEGURO DEUDORES", "FINANCIACIÓN CAPITAL",
        "AMORTIZACIÓN CAPITAL", "SALDO CAPITAL", "Acumulado_financiacion_capital"
    ]]

    # Mensaje
    print("✅ Tabla de amortización generada correctamente.")

except Exception as e:
    # Mensaje de error
    print(f"❌ Error al generar la tabla de amortización: {e}")

✅ Tabla de amortización generada correctamente.


In [136]:
### CALCULAR CUOTAS PAGAS ###

try:    
    # Asegúrate que fecha_data es un datetime
    fecha_data = pd.to_datetime(data.loc[data["nolibra"] == nro_libranza, "fecha_data"].values[0])

    # Calcular diferencia en meses entre fecha_data y corte
    diff = relativedelta(corte_saldo_sistema, fecha_data)
    meses_adicionales = diff.years * 12 + diff.months

    # Cuotas pagadas = las que ya se habían pagado a la fecha_data + las adicionales desde entonces
    cuotas_pagas_base = plazo - (saldo_cierre_mes_0 / cuotadef)
    cuotas_pagas = cuotas_pagas_base + meses_adicionales

    # Mensaje
    print(f"✅ Cuotas pagas calculadas: {cuotas_pagas}")

except Exception as e:
    # Mensaje de error
    print(f"❌ Error al calcular las cuotas pagas: {e}")

✅ Cuotas pagas calculadas: 48.0


In [137]:
### PARAMETROS INCIALES PARA CALCULAR SALDO ACTUAL DEL SISTEMA ###

try:
    # Con esta linea evitar que hayan errores en los siguientes pasos
    df_final["Nº"] = pd.to_numeric(df_final["Nº"], errors="coerce")  
    cuotas_causadas = int(cuotas_causadas)
    cuotas_pagas = int(cuotas_pagas)

    try:
        # Obtener tasas en formato string desde df_final
        tasa_usura_str = df_final.loc[df_final["Nº"] == cuotas_pagas, "TASA DE USURA"].values[0]
        tasa_liq_str = df_final.loc[df_final["Nº"] == cuotas_pagas, "TASA_LIQUIDACIÓN"].values[0]
    except KeyError:
        raise KeyError(f"No se encontraron tasas para cuotas pagas: {cuotas_pagas}. Verifica que la tabla tenga suficientes filas.")

    # Limpiar y convertir a float
    tasa_usura = float(tasa_usura_str.replace('%', '').replace(',', '.'))
    tasa_liq = float(tasa_liq_str.replace('%', '').replace(',', '.'))

    # Elegir tasa mínima y pasar a decimal
    tasa = min(tasa_usura, tasa_liq) / 100
    #print(f"Tasa aplicada: {tasa:.6f}")

    # Extraer saldo capital y convertir a float
    saldo_capital_str = df_final.loc[df_final["Nº"] == cuotas_pagas, "SALDO CAPITAL"].values[0]
    saldo_capital = float(str(saldo_capital_str).replace('.', '').replace(',', '.'))
    #print(f"Saldo capital base: {saldo_capital}")

    # Calcular subtotal obligación mensual por intereses
    dias_mes = calendar.monthrange(datetime.today().year, datetime.today().month)[1]
    subtotal_obligacion_mensual = (saldo_capital * tasa) / dias_mes

    # Calcular días restantes del mes y subtotal obligación proporcional
    dias_restantes = dias_mes - datetime.today().day
    subtotal_obligacion_proporcional = subtotal_obligacion_mensual * dias_restantes

    # Mensaje
    print(f"✅ Parámetros iniciales para calcular saldo actual del sistema obtenidos correctamente.")
    print(f"Tasa usura: {tasa_usura_str}")
    print(f"Tasa liquidación: {tasa_liq_str}")
    print(f"Saldo capital base: {saldo_capital}")
    print(f"Días restantes mes: {dias_restantes}")
    print(f"Subtotal obligación proporcional (intereses prorrateados): {subtotal_obligacion_proporcional:.2f}")

except Exception as e:
    # Mensaje de error
    print(f"❌ Error al obtener los parámetros iniciales: {e}")

✅ Parámetros iniciales para calcular saldo actual del sistema obtenidos correctamente.
Tasa usura: 1,888465%
Tasa liquidación: 1,75000%
Saldo capital base: 5888318.0
Días restantes mes: 23
Subtotal obligación proporcional (intereses prorrateados): 76453.16


In [138]:
### LIMPIAR EL N° ###

try:
    
    # Función para extraer acumulados limpiando formato
    def obtener_acumulado(df, numero):
        valor = df.loc[df["Nº"] == numero, "Acumulado_financiacion_capital"].values
        if len(valor) == 0 or str(valor[0]).strip() == '':
            return 0.0
        valor_limpio = str(valor[0]).replace('.', '').replace(',', '.')
        return float(valor_limpio)
    
    # Mensaje
    print("✅ N° limpiado correctamente.")

except Exception as e:
    # Mensaje de error
    print(f"❌ Error al limpiar el N°: {e}")

✅ N° limpiado correctamente.


In [ ]:
### CALCULAR DATOS DE SALIDA ###

try:
    # Comparar cuotas pagas y causadas para calcular ajuste acumulado
    if cuotas_pagas == cuotas_causadas:
        resultado = 0
    elif cuotas_causadas > cuotas_pagas:
        resultado = obtener_acumulado(df_final, cuotas_causadas) - obtener_acumulado(df_final, cuotas_pagas)
    else:
        resultado = obtener_acumulado(df_final, cuotas_pagas) - obtener_acumulado(df_final, cuotas_causadas)
    #print(f"Diferencia acumulados (resultado): {resultado:.2f}")

    # Calcular base cálculo para días de liquidación (saldo capital a cuotas causadas)
    fila_base_calculo = cuotas_causadas
    valor_base_calc = df_final.loc[df_final["Nº"] == fila_base_calculo, "SALDO CAPITAL"].values
    base_calculo = float(str(valor_base_calc[0]).replace('.', '').replace(',', '.')) if len(valor_base_calc) else 0.0

    # Calcular días para liquidar y resultado proporcional
    fecha_hoy = datetime.today()
    dias_liquidar = (fecha_hoy - corte_saldo_sistema).days
    if fecha_hoy == corte_saldo_sistema:
        resultado_2 = 0
    else:
        resultado_2 = base_calculo * (tasa / 30) * dias_liquidar
    #print(f"Resultado días liquidar (resultado_2): {resultado_2:.2f}")

    # Calcular saldo de seguros proporcional
    saldo_seguros = (cuotas_causadas * seguro_deudor) - (cuotas_pagas * seguro_deudor)
    #print(f"Saldo seguros: {saldo_seguros:.2f}")

    # Calcular saldo actual sumando todos los componentes
    saldo_actual = saldo_capital - resultado + resultado_2 + saldo_seguros
    #print(f"Saldo actual calculado: {saldo_actual:.2f}")

    # Obtener tabla de amortizacion (opcional)
    tabla_amortizacion = df_final.to_dict(orient="records")
    # Convertir a DataFrame para facilitar visualización
    df_tabla_amortizacion = pd.DataFrame(tabla_amortizacion)



    ### CALCULAR MES DE DONDE SE AFECTA LA NOMINA ####
    
    # Diccionario de mes de Ingles a Español
    meses_es = {
        'January': 'Enero', 'February': 'Febrero', 'March': 'Marzo',
        'April': 'Abril', 'May': 'Mayo', 'June': 'Junio',
        'July': 'Julio', 'August': 'Agosto', 'September': 'Septiembre',
        'October': 'Octubre', 'November': 'Noviembre', 'December': 'Diciembre'
    }
    # Fecha actual
    today = pd.Timestamp(datetime.today())
    # Suponiendo que fecha_programada es datetime.date
    if today >= fecha_programada:
        mes = today.strftime("%B")
    else:
        # Obtener el mes anterior (restando 1 día al primer día del mes actual)
        mes_pasado = (today.replace(day=1) - pd.DateOffset(days=1)).strftime("%B")
        mes = mes_pasado
    # Traducir a español y capitalizar
    mes_nomina = meses_es.get(mes, mes).capitalize()






    # Resultado final
    resultado = {
        "nro_libranza": nro_libranza,
        "fecha_inicio": fecha_inicio.strftime("%Y-%m-%d"),
        "cuotas_causadas": cuotas_causadas,
        "cuotas_pagas": cuotas_pagas,
        "saldo actual": saldo_actual,
        "fecha_corte": fecha_programada,
        "mes_nomina": mes_nomina,
    }
    
    # Mensaje
    print("✅ Datos de salida calculados correctamente.")
    print(resultado) # resultados es la informacion objetivo de la simulacion

except Exception as e:
    # Mensaje de error
    print(f"❌ Error al calcular los datos de salida: {e}")

✅ Datos de salida calculados correctamente.
{'nro_libranza': 43827, 'fecha_inicio': '2021-09-30', 'cuotas_causadas': 47, 'cuotas_pagas': 48, 'saldo actual': np.float64(5701061.53875), 'fecha_corte': Timestamp('2025-08-01 00:00:00'), 'mes_nomina': 'Agosto'}
